In [ ]:
# This is R code in juptyer lab, to install R kernel in jupter, open R session inside your oen anaconda environment
# type in the following:
# install.packages("devtools") (fail)
# conda install -c conda-forge r-devtools (Use this)
# conda install -c conda-forge gcc
# devtools::install_github("IRkernel/IRkernel")
# IRkernel::installspec()
# then restart jupterlab
# ref: https://richpauloo.github.io/2018-05-16-Installing-the-R-kernel-in-Jupyter-Lab/

In [ ]:
library(rxnorm)
library(dplyr)
library(data.table)
library(reshape2)
library(stringi)
library(arrow)
library(tidyr)

In [ ]:
path <- '/blue/yonghui.wu/hoyinchan/Data/data2022/'

In [ ]:
amed <- arrow::read_parquet(paste0(path,"med_unified_conversion_before.parquet"))

In [ ]:
amed <- amed %>% select(-'__index_level_0__')

In [ ]:
# Translate ND to RX

nd_table <- amed %>% filter(MEDADMIN_TYPE=='ND') %>% select(-'MEDADMIN_TYPE')

nd_table <- nd_table %>%
  mutate(RX=sapply(MEDADMIN_CODE, rxnorm::from_ndc)) %>% # Apply function to each row and store in new column 'RX'
  rename(ND = MEDADMIN_CODE) # Rename 'MEDADMIN_CODE' to 'ND'

nd_table <- na.omit(nd_table)
nd_table %>% arrow::write_parquet(paste0(path,"med_unified_conversion_nd2rx.parquet"))

In [ ]:
# Translate RX to ATC4th
rx_table <- amed %>% filter(MEDADMIN_TYPE=='RX') %>% select(-'MEDADMIN_TYPE') %>% rename(RX = MEDADMIN_CODE)
rx_table <- unique(c(rx_table$RX, nd_table$RX))
rx_table <- data.frame(RX = rx_table)

rx_table <- rx_table %>% 
    mutate(ATC=sapply(RX, rxnorm::get_atc)) # Apply function to each row and store in new column 'ATC'
rx_table <- na.omit(rx_table)
rx_table %>% arrow::write_parquet(paste0(path,"med_unified_conversion_rx2atc.parquet"))

rx_table_expanded <- rx_table %>%
  unnest(ATC)
rx_table_expanded <- na.omit(rx_table_expanded)
rx_table_expanded %>% arrow::write_parquet(paste0(path,"med_unified_conversion_rx2atc.parquet"))

In [ ]:
length(unique(rx_table_expanded$RX))

In [ ]:
13825/19148